In [2]:
import boto3
import pandas as pd
import ast  
import re
import json
import requests

In [3]:
def get_api_key(file_path):
    with open(file_path, "r") as file:
        return file.read().strip()

In [4]:
def extract_valid_list(response_text):
    response_text = re.sub(r"```[a-zA-Z]*\n", "", response_text).strip("`")
    match = re.search(r"\[.*?\]", response_text, re.DOTALL)
    if match:
        list_text = match.group(0)
        try:
            extracted_list = ast.literal_eval(list_text)
            if isinstance(extracted_list, list):
                return extracted_list
        except Exception:
            pass
    return []

In [5]:
def generate_ner_tags(sentence, api_key):
    url = "https://api.deepinfra.com/v1/openai/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    system_prompt = (
        "You are a large language model that outputs only valid Python lists. "
        "No explanations. No additional text. No Markdown formatting."
    )
    user_prompt = f"""
    You are a highly intelligent language model trained for natural language understanding and named entity recognition.
    Your task is to assign the most suitable NER tags to every token in the following text based on the provided list of possible NER tags.

    Assign NER tags from the set:
    ['B-Further Specification', 'I-Further Specification', 'O', 'I-XOR Gateway',
     'B-XOR Gateway', 'B-Activity Data', 'B-Condition Specification', 'I-Actor',
     'B-AND Gateway', 'I-Activity', 'B-Activity', 'I-Activity Data',
     'I-Condition Specification', 'I-AND Gateway', 'B-Actor']
    to each token in:
    {sentence}

    Output ONLY a Python-style list of NER tags, e.g.:
    ['O', 'O', 'B-Actor', 'I-Actor']

    No explanations. No additional text. Do not include ```python or any Markdown formatting.
    """
    payload = {
        "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        "temperature": 0.0,
        "max_tokens": 1024
    }
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    response_json = response.json()
    response_text = response_json['choices'][0]['message']['content'].strip()
    return extract_valid_list(response_text)

In [6]:
def process_dataset(input_file, output_file, api_key_file):
    api_key = get_api_key(api_key_file)
    dataset = pd.read_csv(input_file)

    def process_row(row):
        sentence = " ".join(ast.literal_eval(row["tokens"]))
        try:
            return generate_ner_tags(sentence, api_key)
        except Exception:
            return []

    dataset["llama-3.1-8B_ner_tags"] = dataset.apply(process_row, axis=1)
    dataset.to_csv(output_file, index=False)
    print(f"Updated dataset saved to {output_file}")

In [7]:
if __name__ == "__main__":
    input_file = "PET-Customized.csv"
    output_file = "PET-Customized-Llama-3.1-8B.csv"
    api_key_file = "./API_Key/DeepInfra.txt"
    process_dataset(input_file, output_file, api_key_file)

Updated dataset saved to PET-Customized-Llama-3.1-8B.csv
